In [1]:
# default_exp node_embedding_evaluation

In [1]:
# export

import csrgraph as cg
import pandas as pd
import numpy as np
from github_search import python_tokens, paperswithcode_tasks

import ast
import astunparse
import csrgraph
import nodevectors

import os
import itertools
import igraph
from sklearn import metrics
import gensim

from operator import itemgetter
import pickle

In [2]:
%cd ..

/home/kuba/Projects/github_search


In [3]:
dependency_records_df = pd.read_csv("output/dependency_records.csv").dropna()

In [4]:
relations = list(
    dependency_records_df.iloc[:, [1, 0]].itertuples(index=False, name=None)
)

In [5]:
# export


def make_igraph(call_records_df):
    vertices = list(
        set(call_records_df["source"].unique()).union(
            call_records_df["destination"].unique()
        )
    )
    edges = [
        (source, destination)
        for (source, destination) in zip(
            call_records_df["source"].values, call_records_df["destination"].values
        )
    ]
    graph = igraph.Graph()
    graph.add_vertices(vertices)
    graph.add_edges(edges)
    return graph

In [6]:
graph = make_igraph(dependency_records_df[["source", "destination"]].dropna())

In [7]:
pickle.dump(graph, open("data/call_igraph.pkl", "wb"))

In [8]:
graph.vcount()

2209729

In [9]:
graph.components().summary()  # ()

'Clustering with 2209729 elements and 1 clusters'

## nodevectors & csrgraph

In [10]:
# export


def get_nodes(call_graph, node_names):
    selected_nodes = call_graph.names[call_graph.names.isin(node_names)]
    return selected_nodes


def get_node_embedding_model_and_results(
    node_embeddings, call_graph, example_nodes, selected_names=None, topk=10
):
    if selected_names is None:
        selected_nodes = call_graph.names
    else:
        selected_nodes = get_node_indices(call_graph, selected_names)

    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(
            node_embeddings[example_nodes.index[[i]]], node_embeddings
        )
        results.append((call_graph.names[np.argsort(distances)[0, :topk]].values))
    results_df = pd.DataFrame(results)
    results_df.index = example_nodes.values
    return results_df.T

In [11]:
dependency_records_df[["source", "destination"]].to_csv(
    "data/dependency_records_raw.csv", index=None
)

In [12]:
dependency_records_df.dropna().shape, dependency_records_df.shape

((9904500, 3), (9904500, 3))

In [13]:
%%time
call_graph = cg.read_edgelist("data/dependency_records_raw.csv", sep=",")
pickle.dump(call_graph, open("call_csrgraph.pkl", "wb"))

CPU times: user 26.1 s, sys: 904 ms, total: 27 s
Wall time: 27 s


In [14]:
example_repo_node_names = [
    "tensorflow/tensorflow",
    "pytorch/fairseq",
    "open-mmlab/mmdetection",
    "huggingface/transformers",
    "google-research/recsim",
    "apache/incubator-mxnet",
    "igul222/improved_wgan_training",
]
example_node_names = [
    "train",
    "fit",
    "test",
    "__init__",
    "DistilBertForMaskedLM",
    "tensorflow",
    "hate-speech-detection",
    "variational-dropout-sparsifies-dnn",
]


def get_node_indices(call_graph, node_names):
    return call_graph.names.isin(node_names)


def get_nodes(call_graph, node_names):
    return call_graph.names[call_graph.names.isin(node_names)]


example_nodes = get_nodes(call_graph, example_node_names)
example_repo_nodes = get_nodes(call_graph, example_repo_node_names)


def get_node_embedding_model_and_results(
    node_embeddings, call_graph, example_nodes, selected_names=None, topk=10
):

    print(len(node_embeddings))
    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(
            node_embeddings[example_nodes.index[[i]]], node_embeddings
        )
        ordered_results = call_graph.names[np.argsort(distances)[0, :]].values
        if not selected_names is None:
            ordered_results = [res for res in ordered_results if res in selected_names]

        results.append(ordered_results[:topk])
    results_df = pd.DataFrame(results)
    results_df.index = example_nodes.values
    return results_df.T

In [15]:
get_nodes(call_graph, ["transformers"])

2052178    transformers
dtype: object

In [16]:
call_graph_nodes = call_graph.nodes()

In [17]:
dependency_records_df.columns

Index(['source', 'destination', 'edge_type'], dtype='object')

In [18]:
call_graph_repo_nodes = dependency_records_df[
    dependency_records_df["edge_type"] == "repo-file"
]["source"].drop_duplicates()

In [19]:
call_graph_repo_nodes

26999               000Justin000/torchdiffeq
27020                  008karan/SincNet_demo
27027       0492wzl/tensorflow_slim_densenet
27095                          08173021/FCOS
27201                     09jvilla/CS234_gym
                         ...                
672087              zzz2010/starganv2_paddle
672098              zzzace2000/autodiagnosis
672134    zzzace2000/dropout-feature-ranking
672177                       zzzer1019/FL_DP
672186           zzzxxxttt/Pytorch_DoReFaNet
Name: source, Length: 26999, dtype: object

In [20]:
dependency_records_df[
    dependency_records_df["source"].str.contains("hate-speech-detection")
]

,source,destination,edge_type
375216,imatge-upc/hate-speech-detection,imatge-upc/hate-speech-detection:test,repo-file
375217,imatge-upc/hate-speech-detection,imatge-upc/hate-speech-detection:train,repo-file
2522322,imatge-upc/hate-speech-detection:train,Timer,file-function
2522323,imatge-upc/hate-speech-detection:test,get_person_class_index,file-function
2522324,imatge-upc/hate-speech-detection:test,heatmaps_to_keypoints,file-function
2522325,imatge-upc/hate-speech-detection:test,keypoints_to_heatmap_labels,file-function
2522326,imatge-upc/hate-speech-detection:test,get_keypoints,file-function
2522327,imatge-upc/hate-speech-detection:test,flip_heatmaps,file-function
2522328,imatge-upc/hate-speech-detection:test,scores_to_probs,file-function
2522329,imatge-upc/hate-speech-detection:test,compute_oks,file-function


%%time
ggvec_model = nodevectors.GGVec(n_components=100, max_epoch=1000)
ggvec_embeddings = ggvec_model.fit_transform(call_graph)

In [21]:
dependency_records_df.shape

(9904500, 3)

In [22]:
%%time
prone_model = nodevectors.ProNE(n_components=200, step=100)
prone_embeddings = prone_model.fit_transform(call_graph)

CPU times: user 1h 12min 2s, sys: 30min 5s, total: 1h 42min 8s
Wall time: 20min 5s


get_node_embedding_model_and_results(ggvec_embeddings, call_graph, example_repo_nodes)

In [23]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

2209729


,apache/incubator-mxnet,huggingface/transformers,igul222/improved_wgan_training,pytorch/fairseq
0,apache/incubator-mxnet,huggingface/transformers,igul222/improved_wgan_training,pytorch/fairseq
1,apache/incubator-mxnet:__init__,papar22/returnn-my-branch,lokiysh/stroopwafel,backdfund/analyzer:store_ds_values
2,apache/incubator-mxnet:dlpack,vvanirudh/CMAX,championway/re3_tracking,babylonhealth/medisim:negative_sampling_from_p...
3,apache/incubator-mxnet:util,IBM/AMLSim,SenWu/emmental-tutorials,blei-lab/ars-reparameterization:gamma_def_gene...
4,apayani/ILP:exp_arithmetic_add,pwollstadt/IDTxl,shvdiwnkozbw/Multi-Source-Sound-Localization,pytorch/fairseq:dynamic_convolution
5,aparecidovieira/Road_extraction,pmulcaire/rosita,sai-prasanna/bert-experiments,backdfund/analyzer:logger
6,apache/incubator-mxnet:test_docker_login,TrackerLB/SPSTracker,chambliss/Multilingual_NER,pytorch/fairseq:lightconv_layer
7,apache/incubator-mxnet:space,STEllAR-GROUP/phylanx,vernamlab/FastSpec,pytorch/fairseq:distributed_fairseq_model
8,apayani/ILP:classify_cora,<ROOT>,amogh7joshi/engagement-detection,babylonhealth/multiverse:experiment_utils
9,apayani/ILP:exp_list_member,STVIR/Grid-R-CNN,wangzheallen/STL-VQA,ml-jku/align-rudder


In [24]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

2209729


,apache/incubator-mxnet,huggingface/transformers,igul222/improved_wgan_training,pytorch/fairseq
0,apache/incubator-mxnet,huggingface/transformers,igul222/improved_wgan_training,pytorch/fairseq
1,apache/incubator-mxnet:__init__,papar22/returnn-my-branch,lokiysh/stroopwafel,backdfund/analyzer:store_ds_values
2,apache/incubator-mxnet:dlpack,vvanirudh/CMAX,championway/re3_tracking,babylonhealth/medisim:negative_sampling_from_p...
3,apache/incubator-mxnet:util,IBM/AMLSim,SenWu/emmental-tutorials,blei-lab/ars-reparameterization:gamma_def_gene...
4,apayani/ILP:exp_arithmetic_add,pwollstadt/IDTxl,shvdiwnkozbw/Multi-Source-Sound-Localization,pytorch/fairseq:dynamic_convolution
5,aparecidovieira/Road_extraction,pmulcaire/rosita,sai-prasanna/bert-experiments,backdfund/analyzer:logger
6,apache/incubator-mxnet:test_docker_login,TrackerLB/SPSTracker,chambliss/Multilingual_NER,pytorch/fairseq:lightconv_layer
7,apache/incubator-mxnet:space,STEllAR-GROUP/phylanx,vernamlab/FastSpec,pytorch/fairseq:distributed_fairseq_model
8,apayani/ILP:classify_cora,<ROOT>,amogh7joshi/engagement-detection,babylonhealth/multiverse:experiment_utils
9,apayani/ILP:exp_list_member,STVIR/Grid-R-CNN,wangzheallen/STL-VQA,ml-jku/align-rudder


In [25]:
dependency_records_df[dependency_records_df["source"] == "mmsegmentation"]

,source,destination,edge_type


In [26]:
"mmsegmentation" in call_graph_repo_nodes

False

In [27]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

2209729


,apache/incubator-mxnet,huggingface/transformers,igul222/improved_wgan_training,pytorch/fairseq
0,apache/incubator-mxnet,huggingface/transformers,igul222/improved_wgan_training,pytorch/fairseq
1,apache/incubator-mxnet:__init__,papar22/returnn-my-branch,lokiysh/stroopwafel,backdfund/analyzer:store_ds_values
2,apache/incubator-mxnet:dlpack,vvanirudh/CMAX,championway/re3_tracking,babylonhealth/medisim:negative_sampling_from_p...
3,apache/incubator-mxnet:util,IBM/AMLSim,SenWu/emmental-tutorials,blei-lab/ars-reparameterization:gamma_def_gene...
4,apayani/ILP:exp_arithmetic_add,pwollstadt/IDTxl,shvdiwnkozbw/Multi-Source-Sound-Localization,pytorch/fairseq:dynamic_convolution
5,aparecidovieira/Road_extraction,pmulcaire/rosita,sai-prasanna/bert-experiments,backdfund/analyzer:logger
6,apache/incubator-mxnet:test_docker_login,TrackerLB/SPSTracker,chambliss/Multilingual_NER,pytorch/fairseq:lightconv_layer
7,apache/incubator-mxnet:space,STEllAR-GROUP/phylanx,vernamlab/FastSpec,pytorch/fairseq:distributed_fairseq_model
8,apayani/ILP:classify_cora,<ROOT>,amogh7joshi/engagement-detection,babylonhealth/multiverse:experiment_utils
9,apayani/ILP:exp_list_member,STVIR/Grid-R-CNN,wangzheallen/STL-VQA,ml-jku/align-rudder


In [28]:
get_node_embedding_model_and_results(
    prone_embeddings,
    call_graph,
    example_repo_nodes,
    selected_names=call_graph_repo_nodes.values,
)

2209729


KeyboardInterrupt: 

In [32]:
prone_embeddings.shape

(2209729, 200)

In [33]:
from gensim.models import KeyedVectors

prone_kv = KeyedVectors(prone_embeddings.shape[1])
prone_kv.add(call_graph.names, prone_embeddings)

In [34]:
prone_kv.save("data/prone_embeddings.bin")

In [34]:
"tensorflow" in prone_kv.vocab.keys()

True

In [35]:
"torch" in prone_kv.vocab.keys()

True

In [101]:
with open("prone_closest_repos.md", "w") as f:
    f.write(
        get_node_embedding_model_and_results(
            prone_embeddings, call_graph, example_repo_nodes
        ).to_markdown()
    )

250891


In [102]:
with open("prone_closest_nodes.md", "w") as f:
    f.write(
        get_node_embedding_model_and_results(
            prone_embeddings, call_graph, example_nodes
        ).to_markdown()
    )

250891


In [103]:
!cat prone_closest_nodes.md

|    | DistilBertForMaskedLM          | __init__                         | fit             | tensorflow                     | test     | train          |
|---:|:-------------------------------|:---------------------------------|:----------------|:-------------------------------|:---------|:---------------|
|  0 | image1                         | devlpl.f:file                    | fit             | image1                         | test     | train          |
|  1 | hand_guide                     | __init__                         | estimate        | hand_guide                     | predict  | evaluate       |
|  2 | hand_dapg_demos:file           | ast_quality_perf_test            | objective       | hand_dapg_demos:file           | infer    | run_training   |
|  3 | hamp                           | factored:file                    | test_prediction | hamp                           | eval     | train_model    |
|  4 | hamming_window                 | time_rhoxy:file                  | t

In [104]:
get_node_embedding_model_and_results(
    prone_embeddings, call_graph, example_nodes
).to_csv(open("prone_closest_nodes.md", "w"))

250891


In [106]:
from gensim.models import poincare

In [107]:
example_edges = list(
    dependency_records_df[["source", "destination"]].itertuples(index=False, name=None)
)

In [114]:
dependency_records_df.shape

(1406481, 3)

In [116]:
model = poincare.PoincareModel(example_edges, negative=256)

In [117]:
# poincare_model = poincare.PoincareModel.load("data/poincare_embeddings.bin")

In [ ]:
%%time
model.train(epochs=2, batch_size=256)

In [ ]:
model.save("data/poincare_embeddings_bigquery.bin")

In [ ]:
poincare_kv = poincare_model.kv

In [ ]:
poincare_kv.most_similar("yolo4", topn=50)

In [ ]:
poincare_kv.most_similar("wgan", topn=50)

In [ ]:
poincare_kv.most_similar("torch")

In [ ]:
poincare_kv.closest_child("mmdetection")

In [ ]:
poincare_kv.most_similar("")

In [ ]:
%%time
model.train(epochs=2, batch_size=64)

In [ ]:
model.kv.descendants("tensorflow")

In [ ]:
model.save("data/poincare_embeddings.bin")

# Hyperbolic embeddings

In [ ]:
import torch

In [ ]:
%load_ext autoreload

In [ ]:
torch.model

In [ ]:
dependency_records_df["edge_type"].value_counts()

## TODO

- expore call graph
- better call graph
- Node2

Exploring call graph

what is the problem?

Graph is not even connected

In [ ]:
i = 2